<a href="https://colab.research.google.com/github/cfong32/key-sentence-extraction/blob/main/exp3_loaddataset_sentencize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q datasets

In [2]:
# import packages
import pandas as pd
from tqdm import tqdm
from datasets import load_dataset
from spacy.lang.en import English

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


# Load CNN/DM dataset

In [3]:
DATASET = 'cnn_dailymail'
CONFIG  = '3.0.0'
SUBSET  = 'test'

dataset = load_dataset(DATASET, CONFIG, split=SUBSET)
df_ds = pd.DataFrame(dataset)
df_ds

,article,highlights,id
0,(CNN)The Palestinian Authority officially beca...,Membership gives the ICC jurisdiction over all...,f001ec5c4704938247d27a44948eebb37ae98d01
1,(CNN)Never mind cats having nine lives. A stra...,"Theia, a bully breed mix, was apparently hit b...",230c522854991d053fe98a718b1defa077a8efef
2,"(CNN)If you've been following the news lately,...",Mohammad Javad Zarif has spent more time with ...,4495ba8f3a340d97a9df1476f8a35502bcce1f69
3,(CNN)Five Americans who were monitored for thr...,17 Americans were exposed to the Ebola virus w...,a38e72fed88684ec8d60dd5856282e999dc8c0ca
4,(CNN)A Duke student has admitted to hanging a ...,Student is no longer on Duke University campus...,c27cf1b136cc270023de959e7ab24638021bc43f
...,...,...,...
11485,Telecom watchdogs are to stop a rip-off that a...,Operators are charging up to 20p a minute - ev...,0ac776a4dc09ca97c136f4314fed4defb48a361a
11486,The chilling reenactment of how executions are...,Bali Nine ringleaders will face the firing squ...,fe89a6a2e28d173e5ad4c6d814c15b95aa969e3f
11487,It is a week which has seen him in deep water ...,Hardy was convicted of domestic abuse against ...,ded2f535cd6ab95d11b5f4ea29bbf2b2d3c55c50
11488,"Despite the hype surrounding its first watch, ...",Apple sold more than 61 million iPhones in the...,30ec5f280eee772a73d181bfc8514defd8026434


# Sentencize one article (by **spacy**)

In [4]:
article = df_ds.iloc[0].article
print(article)

(CNN)The Palestinian Authority officially became the 123rd member of the International Criminal Court on Wednesday, a step that gives the court jurisdiction over alleged crimes in Palestinian territories. The formal accession was marked with a ceremony at The Hague, in the Netherlands, where the court is based. The Palestinians signed the ICC's founding Rome Statute in January, when they also accepted its jurisdiction over alleged crimes committed "in the occupied Palestinian territory, including East Jerusalem, since June 13, 2014." Later that month, the ICC opened a preliminary examination into the situation in Palestinian territories, paving the way for possible war crimes investigations against Israelis. As members of the court, Palestinians may be subject to counter-charges as well. Israel and the United States, neither of which is an ICC member, opposed the Palestinians' efforts to join the body. But Palestinian Foreign Minister Riad al-Malki, speaking at Wednesday's ceremony, sa

In [5]:
spacy_eng_nlp = English()
spacy_eng_nlp.add_pipe("sentencizer")

for i, sent in enumerate(spacy_eng_nlp(article).sents):
    print(i, sent)

0 (CNN)The Palestinian Authority officially became the 123rd member of the International Criminal Court on Wednesday, a step that gives the court jurisdiction over alleged crimes in Palestinian territories.
1 The formal accession was marked with a ceremony at The Hague, in the Netherlands, where the court is based.
2 The Palestinians signed the ICC's founding Rome Statute in January, when they also accepted its jurisdiction over alleged crimes committed "in the occupied Palestinian territory, including East Jerusalem, since June 13, 2014."
3 Later that month, the ICC opened a preliminary examination into the situation in Palestinian territories, paving the way for possible war crimes investigations against Israelis.
4 As members of the court, Palestinians may be subject to counter-charges as well.
5 Israel and the United States, neither of which is an ICC member, opposed the Palestinians' efforts to join the body.
6 But Palestinian Foreign Minister Riad al-Malki, speaking at Wednesday'

# Sentencize all

In [6]:
recs = []
for i, row in tqdm(df_ds.iterrows(), total=len(df_ds)):
    for j, sent in enumerate(spacy_eng_nlp(row['article']).sents):
        recs.append({
            'article_id': row['id'],
            'order'     : j,
            'sentence'  : str(sent),
        })

df_sent = pd.DataFrame(recs)
df_sent

100%|██████████| 11490/11490 [01:05<00:00, 176.21it/s]


,article_id,order,sentence
0,f001ec5c4704938247d27a44948eebb37ae98d01,0,(CNN)The Palestinian Authority officially beca...
1,f001ec5c4704938247d27a44948eebb37ae98d01,1,The formal accession was marked with a ceremon...
2,f001ec5c4704938247d27a44948eebb37ae98d01,2,The Palestinians signed the ICC's founding Rom...
3,f001ec5c4704938247d27a44948eebb37ae98d01,3,"Later that month, the ICC opened a preliminary..."
4,f001ec5c4704938247d27a44948eebb37ae98d01,4,"As members of the court, Palestinians may be s..."
...,...,...,...
387367,b4a1738c4a0acdf3d189264a0927005aa5b856d6,56,"She's pictured here with Nicole, who lives in ..."
387368,b4a1738c4a0acdf3d189264a0927005aa5b856d6,57,"I'm so lucky to be his daughter,' 47-year-old ..."
387369,b4a1738c4a0acdf3d189264a0927005aa5b856d6,58,"Appearing on Ellen last October, Nicole said h..."
387370,b4a1738c4a0acdf3d189264a0927005aa5b856d6,59,Daily Mail Australia has contacted the Kidman ...
